In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

In [3]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.

#IS THIS SAME BETWEEN SENSORS: https://github.com/giswqs/geemap/discussions/672

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

#NOTE 10/6/2022 - This has been updated for Landsat Collection 2 https://www.usgs.gov/landsat-missions/landsat-collection-2

In [4]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('SR_B6'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('SR_B5'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

#Updated 10/6/2022 - Band names changed for Collection 2

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

In [5]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30,
        tileScale = 8 #ADDED 10/6/2022 - make sure it doesn't affect results (see thread below)
    )
    return feature

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature

##FOR 1m DEM:
def dem1Extraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 1 
    )
    return feature

#tileScale: https://gis.stackexchange.com/questions/373250/understanding-tilescale-in-earth-engine

In [6]:
##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem1 = ee.ImageCollection('USGS/3DEP/1m')

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')
Map.addLayer(dem1, dem_params, '1m DEM')

In [7]:
##Calculating mean pixel values for time periods within each year

def monthly_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

## Compute monthly averages
# monthly_sowing_Avg = monthly_Avg (ndvi_sowSeason, years, sowingMonths)

##Months and years are lists

years_ls5 = range(2000, 2012)
years_ls7 = range(2012, 2013)
years_ls8 = range(2013, 2021)
years_dm = range(2000, 2021)

months = range(5,11)
months_daymet = range(1,12)

ls5_collect = ee.ImageCollection(
    'LANDSAT/LT05/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL5).map(maskFLATS).map(addL5ndvi)

ls7_collect = ee.ImageCollection(
    'LANDSAT/LE07/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL5).map(maskFLATS).map(addL5ndvi) ####SHOULD BE maskFLATSL5 - check BANDS!!!

#Note:Landsat datasets migrated to Collection 2

ls8_collect = ee.ImageCollection(
    'LANDSAT/LC08/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATS).map(maskFLATS).map(addL8ndvi)

monthly_ls5 = monthly_Avg(ls5_collect, years = years_ls5)
monthly_ls7 = monthly_Avg(ls7_collect, years = years_ls7)
monthly_ls8 = monthly_Avg(ls8_collect, years = years_ls8)

monthly_ls5.size().getInfo()

##From stackexchange: calculating monthly averages across many years:
#https://gis.stackexchange.com/questions/290892/google-earth-enginesst-by-month-per-year
#https://gis.stackexchange.com/questions/426662/image-collection-monthly-averages-using-geemap-package

12

In [8]:
list_5 = monthly_ls5.toList(monthly_ls5.size())
list_7 = monthly_ls7.toList(monthly_ls7.size())
list_8 = monthly_ls8.toList(monthly_ls8.size())

ls5_2000 = ee.Image(list_5.get(0))
ls5_2001 = ee.Image(list_5.get(1))
ls5_2002 = ee.Image(list_5.get(2))
ls5_2004 = ee.Image(list_5.get(4)) ##SKIP 2003
ls5_2005 = ee.Image(list_5.get(5))
ls5_2006 = ee.Image(list_5.get(6))
ls5_2007 = ee.Image(list_5.get(7))
ls5_2008 = ee.Image(list_5.get(8))
ls5_2009 = ee.Image(list_5.get(9))
ls5_2010 = ee.Image(list_5.get(10))
ls5_2011 = ee.Image(list_5.get(11))

ls7_2012 = ee.Image(list_7.get(0))

ls8_2013 = ee.Image(list_8.get(0))
ls8_2014 = ee.Image(list_8.get(1))
ls8_2015 = ee.Image(list_8.get(2))
ls8_2016 = ee.Image(list_8.get(3))
ls8_2017 = ee.Image(list_8.get(4))
ls8_2018 = ee.Image(list_8.get(5))
ls8_2019 = ee.Image(list_8.get(6))
ls8_2020 = ee.Image(list_8.get(7))

# ls5_2004.getInfo()

In [9]:
##Daymet

def dm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(fc_all)

monthly_dm = dm_Avg(daymet, years = years_dm)

dm_list = monthly_dm.toList(monthly_dm.size())

dm_2000 = ee.Image(dm_list.get(0))
dm_2001 = ee.Image(dm_list.get(1))
dm_2002 = ee.Image(dm_list.get(2))
dm_2004 = ee.Image(dm_list.get(4)) ##SKIP 2003!
dm_2005 = ee.Image(dm_list.get(5))
dm_2006 = ee.Image(dm_list.get(6))
dm_2007 = ee.Image(dm_list.get(7))
dm_2008 = ee.Image(dm_list.get(8))
dm_2009 = ee.Image(dm_list.get(9))
dm_2010 = ee.Image(dm_list.get(10))
dm_2011 = ee.Image(dm_list.get(11))
dm_2012 = ee.Image(dm_list.get(12))
dm_2013 = ee.Image(dm_list.get(13))
dm_2014 = ee.Image(dm_list.get(14))
dm_2015 = ee.Image(dm_list.get(15))
dm_2016 = ee.Image(dm_list.get(16))
dm_2017 = ee.Image(dm_list.get(17))
dm_2018 = ee.Image(dm_list.get(18))
dm_2019 = ee.Image(dm_list.get(19))
dm_2020 = ee.Image(dm_list.get(20))

# dm_2020.getInfo()

In [10]:
##Monthly/yearly extraction - NOTE: Takes a while to run

dem_vals = geemap.ee_to_pandas(demExtraction(dem)) ##10m dataset
# dem_vals = geemap.ee_to_pandas(dem1.map(dem1Extraction).flatten()) ##ONE METER DATASET

# daymet_vals = geemap.ee_to_pandas(monthly_dm.map(rasterExtraction).flatten())

#yrloop = [2000:2020]
#for yr in yrloop: xdm_[yr] = geemap.ee_to_pandas(rasterExtraction(dm_[yr]))
xdm_2000 = geemap.ee_to_pandas(rasterExtraction(dm_2000))
xdm_2001 = geemap.ee_to_pandas(rasterExtraction(dm_2001))
xdm_2002 = geemap.ee_to_pandas(rasterExtraction(dm_2002))
xdm_2004 = geemap.ee_to_pandas(rasterExtraction(dm_2004)) ##SKIP 2003
xdm_2005 = geemap.ee_to_pandas(rasterExtraction(dm_2005))
xdm_2006 = geemap.ee_to_pandas(rasterExtraction(dm_2006))
xdm_2007 = geemap.ee_to_pandas(rasterExtraction(dm_2007))
xdm_2008 = geemap.ee_to_pandas(rasterExtraction(dm_2008))
xdm_2009 = geemap.ee_to_pandas(rasterExtraction(dm_2009))
xdm_2010 = geemap.ee_to_pandas(rasterExtraction(dm_2010))
xdm_2011 = geemap.ee_to_pandas(rasterExtraction(dm_2011))
xdm_2012 = geemap.ee_to_pandas(rasterExtraction(dm_2012))
xdm_2013 = geemap.ee_to_pandas(rasterExtraction(dm_2013))
xdm_2014 = geemap.ee_to_pandas(rasterExtraction(dm_2014))
xdm_2015 = geemap.ee_to_pandas(rasterExtraction(dm_2015))
xdm_2016 = geemap.ee_to_pandas(rasterExtraction(dm_2016))
xdm_2017 = geemap.ee_to_pandas(rasterExtraction(dm_2017))
xdm_2018 = geemap.ee_to_pandas(rasterExtraction(dm_2018))
xdm_2019 = geemap.ee_to_pandas(rasterExtraction(dm_2019))
xdm_2020 = geemap.ee_to_pandas(rasterExtraction(dm_2020))

##Raster vals
# geemap.ee_to_pandas(monthly_ls5.map(rasterExtraction).flatten())
x_2000 = geemap.ee_to_pandas(rasterExtraction(ls5_2000))
x_2001 = geemap.ee_to_pandas(rasterExtraction(ls5_2001))
x_2002 = geemap.ee_to_pandas(rasterExtraction(ls5_2002))
x_2004 = geemap.ee_to_pandas(rasterExtraction(ls5_2004)) ##SKIP 2003
x_2005 = geemap.ee_to_pandas(rasterExtraction(ls5_2005))
x_2006 = geemap.ee_to_pandas(rasterExtraction(ls5_2006))
x_2007 = geemap.ee_to_pandas(rasterExtraction(ls5_2007))
x_2008 = geemap.ee_to_pandas(rasterExtraction(ls5_2008))
x_2009 = geemap.ee_to_pandas(rasterExtraction(ls5_2009))
x_2010 = geemap.ee_to_pandas(rasterExtraction(ls5_2010))
x_2011 = geemap.ee_to_pandas(rasterExtraction(ls5_2011))
x_2012 = geemap.ee_to_pandas(rasterExtraction(ls7_2012))
x_2013 = geemap.ee_to_pandas(rasterExtraction(ls8_2013))
x_2014 = geemap.ee_to_pandas(rasterExtraction(ls8_2014))
x_2015 = geemap.ee_to_pandas(rasterExtraction(ls8_2015))
x_2016 = geemap.ee_to_pandas(rasterExtraction(ls8_2016))
x_2017 = geemap.ee_to_pandas(rasterExtraction(ls8_2017))
x_2018 = geemap.ee_to_pandas(rasterExtraction(ls8_2018))
x_2019 = geemap.ee_to_pandas(rasterExtraction(ls8_2019))
x_2020 = geemap.ee_to_pandas(rasterExtraction(ls8_2020))

display(x_2000)

# ls8_vals = geemap.ee_to_pandas(monthly_ls8.map(rasterExtraction).flatten())

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,SR_ATMOS_OPACITY,SR_CLOUD_QA,SR_B7,QA_PIXEL,ST_CDIST,flats,SR_B1,ST_QA,ST_DRAD,ndvi
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,281.375,8.0,0.053437,5440.0,623.250,0.011662,0.044152,344.875,1276.375,0.290895
1,1,A1,2001,1,1,31.538627,424.416,18,-81.422539,10/17/2001,...,281.375,8.0,0.053437,5440.0,623.250,0.011662,0.044152,344.875,1276.375,0.290895
2,1,A1,2002,1,1,31.538627,259.784,16,-81.422539,10/16/2002,...,281.375,8.0,0.053437,5440.0,623.250,0.011662,0.044152,344.875,1276.375,0.290895
3,1,A1,2004,1,1,31.538627,436.332,18,-81.422539,10/25/2004,...,281.375,8.0,0.053437,5440.0,623.250,0.011662,0.044152,344.875,1276.375,0.290895
4,1,A1,2005,1,1,31.538627,649.300,25,-81.422539,10/24/2005,...,281.375,8.0,0.053437,5440.0,623.250,0.011662,0.044152,344.875,1276.375,0.290895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,8,AX,2012,2,1,31.309053,0.000,1,-81.415765,10/12/2012,...,196.000,4.0,0.066544,5440.0,169.500,0.000172,0.035507,435.875,1210.500,0.482375
2161,8,AX,2012,2,2,31.308965,0.000,1,-81.415714,10/12/2012,...,196.000,4.0,0.066544,5440.0,169.500,0.000172,0.035507,435.875,1210.500,0.482375
2162,9,AX,2017,1,12,31.352050,0.000,1,-81.333824,10/20/2017,...,246.750,1.0,0.051054,5440.0,196.750,0.025228,0.045101,391.375,1439.000,0.346027
2163,9,AX,2017,1,15,31.351793,0.000,1,-81.334335,10/20/2017,...,219.875,1.0,0.057541,5440.0,166.375,0.017631,0.045125,430.625,1271.375,0.329898


In [11]:
##Subsetting dataframes for each year
x_2000 = x_2000[x_2000['Year'] == 2000]
x_2001 = x_2001[x_2001['Year'] == 2001]
x_2002 = x_2002[x_2002['Year'] == 2002]
x_2004 = x_2004[x_2004['Year'] == 2004] ##SKIP 2003
x_2005 = x_2005[x_2005['Year'] == 2005]
x_2006 = x_2006[x_2006['Year'] == 2006]
x_2007 = x_2007[x_2007['Year'] == 2007]
x_2008 = x_2008[x_2008['Year'] == 2008]
x_2009 = x_2009[x_2009['Year'] == 2009]
x_2010 = x_2010[x_2010['Year'] == 2010]
x_2011 = x_2011[x_2011['Year'] == 2011]
x_2012 = x_2012[x_2012['Year'] == 2012]
x_2013 = x_2013[x_2013['Year'] == 2013]
x_2014 = x_2014[x_2014['Year'] == 2014]
x_2015 = x_2015[x_2015['Year'] == 2015]
x_2016 = x_2016[x_2016['Year'] == 2016]
x_2017 = x_2017[x_2017['Year'] == 2017]
x_2018 = x_2018[x_2018['Year'] == 2018]
x_2019 = x_2019[x_2019['Year'] == 2019]
x_2020 = x_2020[x_2020['Year'] == 2020]

landsat_list = [
    x_2000,x_2001,x_2002,x_2004,x_2005,x_2006,x_2007,x_2008,x_2009,x_2010,x_2011,x_2012,x_2013,x_2014,x_2015,x_2016,x_2017,
    x_2018,x_2019,x_2020
]

landsat_extract = pd.concat(landsat_list)

xdm_2000 = xdm_2000[xdm_2000['Year'] == 2000]
xdm_2001 = xdm_2001[xdm_2001['Year'] == 2001]
xdm_2002 = xdm_2002[xdm_2002['Year'] == 2002]
xdm_2004 = xdm_2004[xdm_2004['Year'] == 2004] ##SKIP 2003
xdm_2005 = xdm_2005[xdm_2005['Year'] == 2005]
xdm_2006 = xdm_2006[xdm_2006['Year'] == 2006]
xdm_2007 = xdm_2007[xdm_2007['Year'] == 2007]
xdm_2008 = xdm_2008[xdm_2008['Year'] == 2008]
xdm_2009 = xdm_2009[xdm_2009['Year'] == 2009]
xdm_2010 = xdm_2010[xdm_2010['Year'] == 2010]
xdm_2011 = xdm_2011[xdm_2011['Year'] == 2011]
xdm_2012 = xdm_2012[xdm_2012['Year'] == 2012]
xdm_2013 = xdm_2013[xdm_2013['Year'] == 2013]
xdm_2014 = xdm_2014[xdm_2014['Year'] == 2014]
xdm_2015 = xdm_2015[xdm_2015['Year'] == 2015]
xdm_2016 = xdm_2016[xdm_2016['Year'] == 2016]
xdm_2017 = xdm_2017[xdm_2017['Year'] == 2017]
xdm_2018 = xdm_2018[xdm_2018['Year'] == 2018]
xdm_2019 = xdm_2019[xdm_2019['Year'] == 2019]
xdm_2020 = xdm_2020[xdm_2020['Year'] == 2020]

daymet_list = [
    xdm_2000,xdm_2001,xdm_2002,xdm_2004,xdm_2005,xdm_2006,xdm_2007,xdm_2008,xdm_2009,xdm_2010,xdm_2011,xdm_2012,xdm_2013,
    xdm_2014,xdm_2015,xdm_2016,xdm_2017,xdm_2018,xdm_2019,xdm_2020
]

daymet_extract = pd.concat(daymet_list)

dfx = pd.merge(landsat_extract, dem_vals, how = 'left')

df = pd.merge(dfx, daymet_extract, how = 'right')

display(df)

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,SR_B6,ST_B10,elevation,swe,tmax,srad,tmin,vp,prcp,dayl
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,NaN,NaN,0.303575,0,25.069397,343.573212,14.079397,1765.701416,2.489973,43200.085938
1,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,NaN,NaN,1.106164,0,24.539864,336.004730,15.079507,1874.970703,2.319699,43200.085938
2,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,...,NaN,NaN,NaN,0,24.868795,340.215027,14.635425,1827.156250,2.472329,43200.085938
3,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,NaN,NaN,1.041272,0,24.818274,340.010864,14.757424,1839.845947,2.445726,43200.085938
4,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,NaN,NaN,0.945370,0,24.615616,334.624725,15.242548,1893.765625,2.393671,43200.085938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,0.073398,300.093574,1.277348,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2908,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,0.109336,302.159523,0.411499,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2909,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,0.087468,302.265384,0.302258,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2910,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,0.087468,302.265384,0.101274,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938


In [12]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8') ##make sure no other sensors are being used

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5']-df['SR_B4'])/(df['SR_B5']+df['SR_B4']), \
                      (df['SR_B4']-df['SR_B3'])/(df['SR_B4']+df['SR_B3'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2'], df['SR_B1'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3'], df['SR_B2'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4'], df['SR_B3'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5'], df['SR_B4'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6'], df['SR_B5'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7'], df['SR_B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

display(df)

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_min.csv')
# df.to_csv(out_csv, index = False)

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,Red_band,NIR_band,SWIR1_band,SWIR2_band,savi,wdrvi5,nd_r_g,nd_g_b,nd_swir2_nir,nd_swir2_r
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,0.057757,0.106250,0.087846,0.053437,0.109546,-0.041777,0.010555,0.123125,-0.330733,-0.038859
1,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,0.061580,0.119289,0.085849,0.044585,0.127136,-0.015967,-0.016471,0.162062,-0.455862,-0.160081
2,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,0.065375,0.161518,0.123073,0.071037,0.198399,0.105275,-0.039737,0.216156,-0.389077,0.041504
4,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,0.070992,0.137844,0.107024,0.061508,0.141470,-0.014793,-0.016009,0.127388,-0.382923,-0.071578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,0.045080,0.139123,0.073398,0.036617,0.206174,0.213549,-0.024764,0.394447,-0.583284,-0.103592
2908,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,0.041454,0.205468,0.109336,0.052434,0.329380,0.425001,-0.075481,0.383492,-0.593378,0.116951
2909,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,0.043034,0.163495,0.087468,0.043523,0.255745,0.310250,-0.046433,0.341438,-0.579521,0.005655
2910,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,0.043034,0.163495,0.087468,0.043523,0.255745,0.310250,-0.046433,0.341438,-0.579521,0.005655


In [ ]:
##https://stackoverflow.com/questions/18557860/how-to-create-a-list-with-a-range-of-years

##WITH VARIABLE YEAR IN FILE NAME